# Importing libraries

In [ ]:
import os, glob, json, csv, subprocess, sys, re
from urllib.request import urlopen
from itertools import chain
import urllib.request
from pprint import pprint
from bs4 import BeautifulSoup
from git import *
from subprocess import Popen, PIPE
from os import path
from collections import Counter
import pandas as pd
import itertools

# Defining directories

In [ ]:
userhome = os.path.expanduser('~')
repository = userhome + r'/different-diff/dataset/openjpa/'
analyze_dir = userhome + r'/different-diff/analyze/analyze_openjpa/'

# Defining git command function

In [ ]:
def execute_command(cmd, work_dir):
    #Executes a shell command in a subprocess, waiting until it has completed.
    pipe = subprocess.Popen(cmd, shell=True, cwd=work_dir, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    (out, error) = pipe.communicate()
    return out, error
    pipe.wait()

# Loading candidate bug-fix commit id

In [ ]:
col = ['bug_id','candidate_bugfix_commitID']
bf_cid = pd.read_csv(analyze_dir + '02_extracting_commitid/commitid_of_candidatebugfix.csv')
bf_cid = bf_cid[col]
bf_cid

# Extracting files in each candidate bugfix_commitID

In [ ]:
len(bf_cid)

Extracting the filename in each candidate bug-fix commit id

In [ ]:
file_list = []
for el in range(0,len(bf_cid)):
    sys.stdout.write("\rExtracting bugfix_commitID: {} / {}".format((el+1),len(bf_cid)))
    sys.stdout.flush()
    cmd = "git show --pretty='' --name-only " + bf_cid.iloc[el][1]
    temp = str(execute_command(cmd, repository)).replace("(b'","").split("\\n")[:-1]
    for l, li in enumerate(temp):
        tmp = [bf_cid.iloc[el][0], bf_cid.iloc[el][1], li]
        file_list.append(tmp)
file_list

Saving into CSV file

In [ ]:
with open(analyze_dir + "02_extracting_commitid/file_in_candidatebugfix_commitid.csv", "w") as csvfile:
    fields = ['bug_id', 'candidate_commitID', 'filename']
    writers = csv.writer(csvfile, delimiter = ",")
    writers.writerow(fields)
    for item in file_list:
        writers.writerow(item)
print("File file_in_candidatebugfix_commitid.csv has been created")

Extracting the number of files in each candidate bug-fix commit id

In [ ]:
file_num = []
for el in range(0,len(bf_cid)):
    sys.stdout.write("\rExtracting bugfix_commitID: {} / {}".format((el+1),len(bf_cid)))
    sys.stdout.flush()
    cmd = "git show --pretty='' --name-only " + bf_cid.iloc[el][1] + " | wc -l"
    temp = str(execute_command(cmd, repository)).replace("(b'","").replace(" ","").split("\\n")[:-1]
    for l, li in enumerate(temp):
        num = [bf_cid.iloc[el][0], bf_cid.iloc[el][1], li]
        file_num.append(num)
file_num

In [ ]:
with open(analyze_dir + "02_extracting_commitid/numberof_file_in_candidatebugfix_commitid.csv", "w") as csvfile:
    fields = ['bug_id', 'candidate_bugfix_commitID', 'number_of_files']
    writers = csv.writer(csvfile, delimiter = ",")
    writers.writerow(fields)
    for n in file_num:
        writers.writerow(n)
print("File numberof_file_in_candidatebugfix_commitid.csv has been created")

In [ ]:
df_filenum = pd.read_csv(analyze_dir + "02_extracting_commitid/numberof_file_in_candidatebugfix_commitid.csv")
df_filenum = df_filenum[fields]
df_filenum

In [ ]:
summ = 0
for z in range(0, len(df_filenum)):
    summ = summ + df_filenum.iloc[z][2]
print ("Total number of files for all candidate bug-fix commit id: %i" %summ)